In [10]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

import ccxt as cx
from typing import Any, Optional, List
import ccxt.pro as cxp
from ccxt.base.decimal_to_precision import ROUND_UP
import asyncio
import nest_asyncio
nest_asyncio.apply()

from ccxt.base.exchange import Exchange
from threading import Thread, Event, Lock
from queue import Queue
# from multiprocessing import Queue #as Queue

import qubx
%qubxd

%load_ext autoreload
%autoreload 2

from qubx import lookup
from qubx.core.series import TimeSeries, Bar, Trade, Quote
from qubx.core.strategy import IStrategy, StrategyContext, TriggerEvent
from qubx.core.basics import Instrument, Position, Signal, TransactionCostsCalculator, dt_64
from qubx.impl.ccxt_connector import CCXTConnector, CtrlChannel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Common

In [ ]:
lookup.instruments['KRAKEN.F:BTC/USD']

In [34]:
instr = lookup.instruments.find('BINANCE.UM', 'BTC', 'USDT')
instr

Instrument(symbol='BTCUSDT', market_type='CRYPTO', exchange='BINANCE.UM', base='BTC', quote='USDT', margin_symbol='USDT', min_tick=0.1, min_size_step=0.001, min_size=0.001, futures_info=FuturesInfo(contract_type='PERPETUAL', delivery_date=datetime.datetime(2100, 12, 25, 10, 0), onboard_date=datetime.datetime(2019, 9, 25, 11, 0), contract_size=1.0, maint_margin=2.5, required_margin=5.0, liquidation_fee=0.0125), _aux_instrument=None)

In [39]:
instr = lookup.instruments.find('KRAKEN', 'BTC', 'ETH')
instr

Instrument(symbol='ETH/BTC', market_type='spot', exchange='KRAKEN', base='ETH', quote='BTC', margin_symbol=None, min_tick=1e-05, min_size_step=1e-05, min_size=1e-08, futures_info=None, _aux_instrument=None)

In [38]:
lookup.find_aux_instrument_for(instr, 'USD')

Instrument(symbol='BTC/USD', market_type='spot', exchange='KRAKEN', base='BTC', quote='USD', margin_symbol=None, min_tick=0.1, min_size_step=0.1, min_size=1e-08, futures_info=None, _aux_instrument=None)

In [46]:
bu = cxp.binanceusdm()
bs = cxp.binance()
ks = cxp.kraken()
kf = cxp.krakenfutures()

# Get OHLC [TODO]

In [47]:
bs_markets = await bs.load_markets()
bu_markets = await bu.load_markets()
kf_markets = await kf.load_markets()
ks_markets = await ks.load_markets()

In [53]:
bu.parse_timeframe('1m')

60

In [ ]:
interval = bu.parse_timeframe('1m') * 1000
orderbook_max_levels_limit = 100
since = bu.round_timeframe('1m', bu.milliseconds(), ROUND_UP) - orderbook_max_levels_limit * interval
await bu.fetch_ohlcv('ETH/USDT', '1m', since=since, limit=orderbook_max_levels_limit)

In [ ]:
# await bnc1.fetch_ohlcv('ETH/USDT', '1m', since=since, limit=limit)

In [57]:
cxc = CCXT_connector('binance')

# Listen to data

In [8]:
class RunThread(Thread):
    def __init__(self, channel: CtrlChannel):
        self.result = None
        self.ch = channel
        self.loops = []
        super().__init__()

    def add(self, func, *args, **kwargs):
        self.loops.append(func(self.ch, *args, **kwargs))

    async def run_loop(self):
        self.result = await asyncio.gather(*self.loops)

    def run(self):
        self.ch.control.set()
        asyncio.run(self.run_loop())

    def stop(self):
        self.ch.control.clear()
        self.ch.queue.put((None, None)) # send sentinel


async def listen_to_trades(channel: CtrlChannel, exchange: Exchange, symbols: List[str]):
    method = 'watchTrades'
    if exchange.has[method]:
        # lock = channel.lock
        while channel.control.is_set():
            try:
                # c_time = exchange.iso8601(exchange.milliseconds())
                trades = await exchange.watch_trades_for_symbols(symbols)
                # lock.acquire()
                for trade in trades:
                    # trade = trades[-1]
                    t_ns = trade['timestamp'] * 1_000_000 # this is trade time 
                    s = trade['symbol']
                    info = trade['info']
                    price = trade['price']
                    m = info['m']
                    amnt = trade['amount'] # in base currency
                    tas = Trade(t_ns, price, amnt, int(not m), int(trade['id']))
                    channel.queue.put((s, tas))
                # lock.release()
                # print(trade, flush=True)
                # print(f"\t[{c_time}] {s} : {str(tas)}", flush=True)
            except Exception as e:
                print(type(e).__name__, str(e), flush=True)
                await exchange.close()
                raise e
    else:
        raise Exception(exchange.id + ' ' + method + ' is not supported or not implemented yet')


async def listen_to_orderbook(channel: CtrlChannel, exchange: Exchange, symbols: List[str], orderbook_max_levels_limit=5):
    lock = channel.lock
    while channel.control.is_set():
        try:
            orderbook = await exchange.watch_order_book_for_symbols(symbols, limit=orderbook_max_levels_limit)
            # - check what this time is ?
            t_ns = orderbook['timestamp'] * 1_000_000
            s = orderbook['symbol']
            b, a = orderbook['bids'], orderbook['asks']
            q = Quote(t_ns, b[0][0], a[0][0], b[0][1], a[0][1])
            # lock.acquire()
            channel.queue.put((s, q))
            # lock.release()
            # c_time = exchange.iso8601(exchange.milliseconds())
            # print(f"[{c_time}] {s} : {str(q)}", flush=True)
        except Exception as e:
            print(type(e).__name__, str(e), flush=True)
            await exchange.close()
            raise e
 

async def receiver(channel: CtrlChannel):
    sers = {}
    print("START LISTENING", flush=True)
    T, Q = 0, 0
    while channel.control.is_set():
        s, data = channel.queue.get()
        if isinstance(data, Quote):
            if not (Q % 100):
                print(s, data, flush=True)
                Q = 0
            Q += 1
        if isinstance(data, Trade):
            if s not in sers:
                sers[s] = TimeSeries(s, '1Min')
            ts = sers[s]
            ts.update(data.time, data.price)
            if not (T % 100):
                print(s, data, flush=True)
                T = 0
            T += 1
        if isinstance(data, Bar):
            # print(f"{s} {pd.Timestamp(data.time, unit='ns')}: {data}", flush=True)
            if s not in sers:
                sers[s] = {}
            sers[s][data.time] = data

    print(f"STOP LISTENING: {str(sers)}", flush=True)
    return sers 

In [108]:
def prefetch_ohlcs(exchange: Exchange, symbol: str, timeframe: str, nbarsback: int):
    assert nbarsback > 1
    start = ((pd.Timestamp('now', tz='UTC') - nbarsback * pd.Timedelta(timeframe)).asm8.item()//1000000) if nbarsback > 1 else None 
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since=start, limit=nbarsback + 1)
    return ohlcv

async def listen_to_ohlcv(channel: CtrlChannel, exchange: Exchange, 
                          symbol: List[str], timeframe: str, nbarsback: int):
    # - check if we need to load initial 'snapshot'
    if nbarsback > 1:
        ohlcv = await prefetch_ohlcs(exchange, symbol, timeframe, nbarsback)
        for oh in ohlcv:
            b = Bar(oh[0] * 1_000_000, oh[1], oh[2], oh[3], oh[4], oh[6], oh[7])
            channel.queue.put((symbol, b))

    while channel.control.is_set():
        try:
            ohlcv = await exchange.watch_ohlcv(symbol, timeframe)
            for oh in ohlcv:
                b = Bar(oh[0] * 1000000, oh[1], oh[2], oh[3], oh[4], oh[6], oh[7])
                channel.queue.put((symbol, b))
        except Exception as e:
            print(type(e).__name__, str(e), flush=True)
            await exchange.close()
            raise e

In [7]:
# ohlcv = Binance().fetch_ohlcv('BTC/USDT', '1h')

In [72]:
cc = CCXTConnector('binance')

In [73]:
cc.time()

numpy.datetime64('2024-03-27T10:35:37.566759000')

In [74]:
class ZeroStrat(IStrategy):
    param1: int = 0

    def on_init(self):
        pass

    def process_event(self, time: dt_64, event: TriggerEvent) -> Optional[List[Signal]]:
        print(f"I'm Here param1 = {self.param1}!")
        return None

In [75]:
ctx = StrategyContext(
    ZeroStrat(), dict(param1 = 1), cc, cc, 
    instruments=lookup.instruments[r'BINANCE:(BTC|ETH|LTC)USDT'],
    subscription_params={'timeframe': '5Min'},
    fees_spec='vip0_usdt', base_currency='USDT'
)

2024-03-27 12:35:40.079 [ ℹ️ ] Set param1 -> 1
2024-03-27 12:35:40.080 [ ℹ️ ] Subscribing on ohlc data using {'timeframe': '5Min'} for 
['BTCUSDT', 'ETHUSDT', 'LTCUSDT'] 


In [78]:
await cc.exch.close()

<hr/>

In [ ]:
chan = CtrlChannel('ticks')
reader = RunThread(chan)
reader.add(receiver)
reader.start()

In [17]:
symbs = ['1INCHUSDT', 'AGLDUSDT', 'APEUSDT', 'ARBUSDT', 'AUCTIONUSDT', 'BANDUSDT', 'BATUSDT', 'C98USDT', 'COMPUSDT', 'CYBERUSDT',
        'DYDXUSDT', 'ENSUSDT', 'FXSUSDT', 'HFTUSDT', 'ICXUSDT', 'IOSTUSDT', 'IOTAUSDT', 'IOTXUSDT', 'LDOUSDT', 'LQTYUSDT', 'LTCUSDT',
        'MASKUSDT', 'MBOXUSDT', 'MINAUSDT', 'MTLUSDT', 'OGNUSDT', 'OPUSDT', 'PEOPLEUSDT', 'POLSUSDT', 'PYRUSDT', 'QNTUSDT', 'RADUSDT',
        'RDNTUSDT', 'RIFUSDT', 'RLCUSDT', 'SEIUSDT', 'SPELLUSDT', 'SSVUSDT', 'STGUSDT', 'STXUSDT', 'SUIUSDT', 'UMAUSDT', 'WAVESUSDT']

In [18]:
chan = cc.subscribe('ohlc', symbs, '5m', 10)

In [19]:
reader = RunThread(chan)
reader.add(receiver)
reader.start()

START LISTENING


In [21]:
reader.stop()

In [85]:
# cxp.binanceqv = BinanceQV

In [87]:
# bu = cxp.binanceusdm()
# bu = cxp.binance()
# bu = BinanceQV()
bu = getattr(cxp, 'binanceqv')()
t1 = RunThread(chan)
for s in symbs:
    t1.add(listen_to_ohlcv, bu, s, '15m', 10)

# t1.add(listen_to_ohlcv, bu, 'BTCUSDT', '5m', 10)
# t1.add(listen_to_ohlcv, bu, 'ETHUSDT', '5m', 10)
# t1.add(listen_to_ohlcv, bu, 'LTCUSDT', '5m', 10)
# t1.add(listen_to_ohlcv, bu, 'SOLUSDT', '5m', 10)
# t1.add(listen_to_trades, bu, ['BTCUSDT', 'ETHUSDT', 'SOLUSDT'])
# t1.add(listen_to_orderbook, bu, ['BTCUSDT', 'ETHUSDT', 'SOLUSDT'])
t1.start()

In [ ]:
bu.ohlcvs

In [102]:
t1.stop()
bu.close()

STOP LISTENING: {'1INCHUSDT': {1711456200000000000: {o:0.613800 | h:0.614700 | l:0.613400 | c:0.613600 | v:15241.534540}, 1711457100000000000: {o:0.613600 | h:0.614000 | l:0.611300 | c:0.613500 | v:37287.844450}, 1711458000000000000: {o:0.613600 | h:0.613900 | l:0.610600 | c:0.613700 | v:25501.184880}, 1711458900000000000: {o:0.613400 | h:0.614500 | l:0.611100 | c:0.611900 | v:141364.905710}, 1711459800000000000: {o:0.611600 | h:0.611600 | l:0.603600 | c:0.608800 | v:113335.154420}, 1711460700000000000: {o:0.608800 | h:0.611800 | l:0.605100 | c:0.607600 | v:137946.043000}, 1711461600000000000: {o:0.607500 | h:0.611600 | l:0.604000 | c:0.610500 | v:117728.467580}, 1711462500000000000: {o:0.610300 | h:0.610800 | l:0.600000 | c:0.600000 | v:142956.323210}, 1711463400000000000: {o:0.600000 | h:0.603600 | l:0.594500 | c:0.603200 | v:338717.297250}, 1711464300000000000: {o:0.602900 | h:0.607900 | l:0.602700 | c:0.607900 | v:64653.737190}}, 'AGLDUSDT': {1711456200000000000: {o:1.597000 | h:1.

<coroutine object Exchange.close at 0x000001D60929CD00>

In [103]:
len(reader.result[0]), len(symbs)

(43, 43)

In [104]:
reader.result[0]

{'1INCHUSDT': {1711456200000000000: {o:0.613800 | h:0.614700 | l:0.613400 | c:0.613600 | v:15241.534540},
  1711457100000000000: {o:0.613600 | h:0.614000 | l:0.611300 | c:0.613500 | v:37287.844450},
  1711458000000000000: {o:0.613600 | h:0.613900 | l:0.610600 | c:0.613700 | v:25501.184880},
  1711458900000000000: {o:0.613400 | h:0.614500 | l:0.611100 | c:0.611900 | v:141364.905710},
  1711459800000000000: {o:0.611600 | h:0.611600 | l:0.603600 | c:0.608800 | v:113335.154420},
  1711460700000000000: {o:0.608800 | h:0.611800 | l:0.605100 | c:0.607600 | v:137946.043000},
  1711461600000000000: {o:0.607500 | h:0.611600 | l:0.604000 | c:0.610500 | v:117728.467580},
  1711462500000000000: {o:0.610300 | h:0.610800 | l:0.600000 | c:0.600000 | v:142956.323210},
  1711463400000000000: {o:0.600000 | h:0.603600 | l:0.594500 | c:0.603200 | v:338717.297250},
  1711464300000000000: {o:0.602900 | h:0.607900 | l:0.602700 | c:0.607900 | v:64653.737190}},
 'AGLDUSDT': {1711456200000000000: {o:1.597000 | h

In [ ]:
# reader.result[0]['SOL/USDT:USDT'].plot()
reader.result[0]['ETH/USDT:USDT'].plot()

In [ ]:
reader.result[0]['BTC/USDT:USDT'].plot()